In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt          # plotting
from scipy.sparse import csr_matrix      # sparse matrices
%matplotlib inline

In [2]:
wiki = pd.read_csv('people_wiki.csv')

In [3]:
wiki.head(10)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...
6,<http://dbpedia.org/resource/Aaron_LaCrate>,Aaron LaCrate,aaron lacrate is an american music producer re...
7,<http://dbpedia.org/resource/Trevor_Ferguson>,Trevor Ferguson,trevor ferguson aka john farrow born 11 novemb...
8,<http://dbpedia.org/resource/Grant_Nelson>,Grant Nelson,grant nelson born 27 april 1971 in london also...
9,<http://dbpedia.org/resource/Cathy_Caruth>,Cathy Caruth,cathy caruth born 1955 is frank h t rhodes pro...


In [4]:
wiki.iloc[0,2]

'digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continued to play victorian football league vfl football with the northern bullants carltons vfla

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [7]:
word_count = load_sparse_csr('people_wiki_word_count.npz')

In [8]:
word_count

<59071x547979 sparse matrix of type '<type 'numpy.int64'>'
	with 10379283 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(word_count)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [10]:
print wiki[wiki['name'] == 'Barack Obama']

                                              URI          name  \
35817  <http://dbpedia.org/resource/Barack_Obama>  Barack Obama   

                                                    text  
35817  barack hussein obama ii brk husen bm born augu...  


In [14]:
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10) # 1st

In [16]:
distances

array([[  0.        ,  33.07567082,  34.39476704,  36.15245497,
         36.16628264,  36.33180425,  36.40054945,  36.49657518,
         36.63331817,  36.95943723]])

In [17]:
indices

array([[35817, 24478, 28447, 35357, 14754, 13229, 31423, 22745, 36364,
         9210]])

In [20]:
distances.shape, indices.shape

((1, 10), (1, 10))

In [32]:
pd.DataFrame(distances.T, columns=['distances'])

,distances
0,0.000000
1,33.075671
2,34.394767
3,36.152455
4,36.166283
5,36.331804
6,36.400549
7,36.496575
8,36.633318
9,36.959437


In [36]:
wiki.iloc[indices[0].tolist(),:]['name']

35817                  Barack Obama
24478                     Joe Biden
28447                George W. Bush
35357              Lawrence Summers
14754                   Mitt Romney
13229              Francisco Barrio
31423                Walter Mondale
22745    Wynn Normington Hugh-Jones
36364                    Don Bonker
9210                   Andy Anstett
Name: name, dtype: object

In [49]:
# now we can load it back
with open('people_wiki_map_index_to_word.json','r') as f:
    map_index_to_word = json.load(f)

In [50]:
def unpack_dict(matrix, map_index_to_word):
    #table = list(map_index_to_word.sort('index')['category'])
    # if you're not using SFrame, replace this line with
    table = sorted(map_index_to_word, key=map_index_to_word.get)
    
    
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} \
               for i in xrange(num_doc) ]

wiki['word_count'] = unpack_dict(word_count, map_index_to_word)

In [52]:
wiki.head()

,URI,name,text,word_count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{u'selection': 1, u'carltons': 1, u'being': 1,..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{u'precise': 1, u'thomas': 1, u'they': 1, u'di..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{u'just': 1, u'issued': 1, u'mainly': 1, u'nom..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{u'all': 1, u'bauforschung': 1, u'just': 1, u'..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{u'legendary': 1, u'gangstergenka': 1, u'legen..."


In [55]:
def top_words(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    row = wiki[wiki['name'] == name]
    word_count_table = row[['word_count']].stack('word_count', new_column_name=['word','count'])
    return word_count_table.sort('count', ascending=False)

obama_words = top_words('Barack Obama')
print obama_words

barrio_words = top_words('Francisco Barrio')
print barrio_words

TypeError: stack() got an unexpected keyword argument 'new_column_name'

In [56]:
row = wiki[wiki['name'] == 'Barack Obama']

In [57]:
row

,URI,name,text,word_count
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...,"{u'operations': 1, u'represent': 1, u'office':..."


In [58]:
row[['word_count']]

,word_count
35817,"{u'operations': 1, u'represent': 1, u'office':..."
